In [ ]:
#v1_layer1

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns

import requests
import ccxt
from ccxt import InvalidOrder

import subprocess
import time

import sys

def send_telegram_message(message):
    bot_token = 
    chat_id = 
    url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    params = {'chat_id': chat_id, 'text': message}
    response = requests.post(url, json=params)

def fetch_top_crypto_close_prices(top, interval):
    binance_endpoint_24hr = "https://fapi.binance.com/fapi/v1/ticker/24hr"

    def fetch_kline_data(symbol):
        endpoint = f"https://fapi.binance.com/fapi/v1/klines?symbol={symbol}&limit=1500&interval={interval}"
        response = requests.get(endpoint)
        return response.json() if response.status_code == 200 else None

    response = requests.get(binance_endpoint_24hr)
    binance_response_24hr = response.json()

    layer1_coin_list = ['ETH', 'SUI', 'ADA', 'AVAX', 'BNB', 'TIA', 'ALT', 'CFX',
                        'ETC', 'SEI', 'TRX', 'DOT', '1000LUNC', 'ONDO', 'ATOM', 'APT',
                        'HBAR', 'ICP', 'FTM', 'EOS', 'RONIN', 'STX', 'LSK', 'CELO', 'CHZ',
                        'AMB', 'LUNA2', 'NEO', 'ROSE', 'IOTA', 'ALGO', 'XEM', 'EGLD', 'KSMU',
                        'WAVES', 'XTZU', 'FLOW', 'VETU', 'MOVR', 'NTRN', 'KAVA', 'POLYX',
                        'ZIL', 'QTUM', 'IOTX', 'ONT', 'KLAY', 'IOST', 'ICX', 'ONGU']

    layer1_coin_list_with_usdt = [coin + 'USDT' for coin in layer1_coin_list]

    # Filter out 'BTCUSDT' and 'SOLUSDT'
    # filtered_response = [item for item in binance_response_24hr if item['symbol'] not in ['BTCUSDT', 'SOLUSDT', 'BTCUSDC']]

    filtered_response = [item for item in binance_response_24hr if item['symbol'] in layer1_coin_list_with_usdt]

    price_change_percents = np.array([float(item['priceChangePercent']) for item in filtered_response])
    mean_price_change_percent = np.mean(price_change_percents)
    std_dev_price_change_percent = np.std(price_change_percents)
    symbols_within_2std_dev = [item for item in filtered_response
                                if abs(float(item['priceChangePercent']) - mean_price_change_percent) <= 1.5 * std_dev_price_change_percent
                                ]

    weighted_volumes = {item['symbol']: float(item['weightedAvgPrice']) * float(item['volume'])
                        for item in symbols_within_2std_dev}

    top_symbols = sorted(weighted_volumes, key=weighted_volumes.get, reverse=True)[:top]
    
    df_list = [pd.DataFrame([float(kline[4]) for kline in fetch_kline_data(symbol)],
                            index=[datetime.fromtimestamp(int(kline[0]) / 1000) for kline in fetch_kline_data(symbol)],
                            columns=[symbol])
               for symbol in top_symbols if fetch_kline_data(symbol)]

    return pd.concat(df_list, axis=1)

def clean_dataframe(df, missing_threshold=0.3):
    missing_frac = df.isnull().mean()
    drop_list = missing_frac[missing_frac > missing_threshold].index
    df_cleaned = df.drop(labels=drop_list, axis=1)
    df_cleaned = df_cleaned.fillna(method='ffill')
    return df_cleaned
    
def PCWeights(pca):
    """
    Principal Compoments Weights
    """
    weights = pd.DataFrame()
    
    for i in range(len(pca.components_)):
        weights["weights_{}".format(i)] = pca.components_[i] / sum(pca.components_[i]
                                                                   )
        
    weights = weights.values.T

    return weights

# def PCWeights_long():
#     weights = PCWeights()
#     weights_long = np.zeros_like(weights)
    
#     for i, arr in enumerate(weights):
#         arr[arr < 0] = 0  # Set negative values to 0
#         if arr.sum() > 0:
#             arr /= arr.sum()  # Normalize to sum up to 1
#         weights_long[i] = arr

#     return weights_long

def sharpe_ratio(ts_returns, periods_per_year = 1): # 이거 일봉이 아니라 30분봉이라 365는 사실 아님
    
    n_years = ts_returns.shape[0]/periods_per_year
    annualized_return = np.power(np.prod(1 + ts_returns), (1 / n_years)
                                 ) - 1
    annualized_vol = ts_returns.std() * np.sqrt(periods_per_year)
    annualized_sharpe = annualized_return / annualized_vol
    return annualized_return, annualized_vol, annualized_sharpe

def FindOptimalPortfolio(pca, scaled_coin, X_Train_Raw):
    
    n_portfolios = len(pca.components_)
    annualized_ret = np.array([0.] * n_portfolios)
    sharpe_metric = np.array([0.] * n_portfolios)
    annualized_vol = np.array([0.] * n_portfolios)
    highest_sharpe = 0
    coin_tickers = scaled_coin.columns.values
    n_tickers = len(coin_tickers)
    PCs = pca.components_
    
    for i in range(n_portfolios):
        
        pc_w = PCs[i] / sum(PCs[i]
                           )

        # #여기서부터 롱밖에 없다는 관점으로 내가 코드 추가함, 숏 다 없애고 롱 비중은 스케일링
        # pc_w[pc_w < 0] = 0
        # pc_w /= pc_w.sum()
        # #여기까지    
        
        eigen_prtfi = pd.DataFrame(data = {"weights": pc_w.squeeze()*100}, 
                                   index = coin_tickers)
        eigen_prtfi.sort_values(by = ["weights"],
                                ascending = False,
                                inplace = True)
        eigen_prti_returns = np.dot(X_Train_Raw.loc[:, eigen_prtfi.index], 
                                    pc_w)
        eigen_prti_returns = pd.Series(eigen_prti_returns.squeeze(),
                                       index = X_Train_Raw.index)
        er, vol, sharpe = sharpe_ratio(eigen_prti_returns)
        annualized_ret[i] = er
        annualized_vol[i] = vol
        sharpe_metric[i] = sharpe
        sharpe_metric = np.nan_to_num(sharpe_metric)
        
    # HOW TO FIND A PORTFOLIO with the HIGHEST Sharpe Ratio
    
    highest_sharpe = np.argmax(sharpe_metric)
        
    results = pd.DataFrame(data = {"Return": annualized_ret, "Vol": annualized_vol, "Sharpe": sharpe_metric}
                           )
    results.dropna(inplace = True)
    results.sort_values(by = ["Sharpe"],
                        ascending = False,
                        inplace = True)
    return results.head(15)
        
def VisualizeEigen(weights, coin_tickers,
                   portfolio = pd.DataFrame()):
    
    portfolio = pd.DataFrame(data = {"weights": weights.squeeze()*100},
                             index = coin_tickers)
    portfolio.sort_values(by = ["weights"],
                          ascending = False,
                          inplace = True)
    return portfolio

def allocation_df(visualize_eigen, capital):

    df = visualize_eigen
    df['Position'] = df['weights'].apply(lambda x: 'Long' if x > 0 else 'Short')
    #df['Position'] = df['weights'].apply(lambda x: 'Short' if x > 0 else 'Long')
    total_weights = df['weights'].abs().sum()
    df['Capital_Allocation'] = (df['weights'].abs() / total_weights) * capital  # Assuming $1000 investment

    return df

# Main execution
def main(top, interval, capital, leverage, df_show = False):

    def execute_orders(df, api_key, api_secret, leverage):
    # Set up the exchange
        exchange = ccxt.binance({
            'apiKey': api_key,
            'secret': api_secret,
            'enableRateLimit': True,
            'options': {'defaultType': 'future'}
        })

        # Iterate over the DataFrame and create orders
        for symbol, row in df.iterrows():
            position = row['Position']
            capital_allocation = row['Capital_Allocation']

            # Get current price for the symbol
            try:
                ticker = exchange.fetch_ticker(symbol)
                current_price = ticker['last']

                # Calculate the amount for the order
                amount = capital_allocation * leverage / current_price

                # Correct the amount precision
                precised_amount = exchange.amount_to_precision(symbol, amount)

                # Determine order side based on 'Position'
                side = 'buy' if position == 'Long' else 'sell'

                # Create a market order
                try:
                    resp = exchange.set_leverage(leverage=leverage, symbol=symbol)
                    order = exchange.create_order(symbol, 'market', side, precised_amount)
                    print(f"{position} order executed for {symbol}: {order}")
                except Exception as e:
                    print(f"Error executing {position.lower()} order for {symbol}: {e}")
            except InvalidOrder as io:
                print(f"Invalid order for {symbol}, skipping to next. Error: {io}")
            except Exception as e:
                print(f"General error for {symbol}, skipping to next. Error: {e}")
    
    xBinance_KEY = 
    xBinance_SECRET = 

    coin = clean_dataframe(fetch_top_crypto_close_prices(top, interval))

    Daily_Linear_Returns = coin.pct_change(1)
    Daily_Linear_Returns = Daily_Linear_Returns[Daily_Linear_Returns.apply(lambda x:(x - x.mean()
                                                                                    ).abs() < (3 * x.std()
                                                                                            )
                                                                        ).all(1)
                                                ]

    scaler = StandardScaler().fit(Daily_Linear_Returns)
    scaled_coin = pd.DataFrame(scaler.fit_transform(Daily_Linear_Returns),
                            columns = Daily_Linear_Returns.columns,
                            index = Daily_Linear_Returns.index)

    Daily_Linear_Returns.dropna(how = "any",
                                inplace = True)

    scaled_coin.dropna(how = "any",
                    inplace = True)

    coin_tickers = scaled_coin.columns.values
    n_tickers = len(coin_tickers)

    X_Train = scaled_coin
    X_Train_Raw = Daily_Linear_Returns

    pca = PCA()

    PrincipalComponent = pca.fit(X_Train)

    df = FindOptimalPortfolio(pca, scaled_coin, X_Train_Raw)

    positive_return_df_excluding_row_0 = df[(df['Return'] > 0) & (df.index != 0)]

        # Check if the DataFrame is empty
    if positive_return_df_excluding_row_0.empty:
        # print("No rows meet the specified conditions. Adjust the filters or check the input data.")
        return

    else:
        highest_return_row_with_conditions = positive_return_df_excluding_row_0['Sharpe'].idxmax()

        target_weight_int = highest_return_row_with_conditions

        portfoilio_weight_df = VisualizeEigen(weights = PCWeights(pca)[target_weight_int], coin_tickers = coin_tickers)

        execution_df = allocation_df(portfoilio_weight_df, capital)

        execution_df.to_csv("df_1.csv")

        execute_orders(execution_df, xBinance_KEY, xBinance_SECRET, leverage = leverage)
        send_telegram_message('🟢 LAYER1 Position entered')

        # 이거 꼭 고쳐야함
        time.sleep(5)
        subprocess.Popen(["python", "pca_execution_L1_v2.py"])
        sys.exit()

        if df_show:
            return df, execution_df
            
        return execution_df

while True:
    result = main(top=20, interval='1m', capital=10000, leverage = 5, df_show = False)

    if not result.empty:
        break

    print("Condition not met. Retrying in 5 minutes...")

    time.sleep(300)
